In [1]:
from evo.tools import file_interface
from evo.core import sync, trajectory
import numpy as np
import os
from evo.core import lie_algebra as lie
from scipy.spatial.transform import Rotation
# from matplotlib import pyplot as plt

In [2]:
DIR="/media/prakhar/BIG_BAG/Capstone/habitat/anand_dec9/"
traj_ref1 = file_interface.read_tum_trajectory_file(DIR+"frl_apartment_0_1607517702/frl_apartment_0_1607517702_gt2.txt")
traj_ref2 = file_interface.read_tum_trajectory_file(DIR+"frl_apartment_0_1607520043/frl_apartment_0_1607520043_gt2.txt")
traj_ref3 = file_interface.read_tum_trajectory_file(DIR+"frl_apartment_1_1607520143/frl_apartment_1_1607520143_gt2.txt")

gt_start = [traj_r.timestamps[0] for traj_r in [traj_ref1, traj_ref2, traj_ref3]]

In [3]:
gt_start

[1607517702.49489, 1607520043.211885, 1607520143.727371]

In [4]:
os3path = "/media/prakhar/BIG_BAG/Capstone/habitat/anand_dec9/f_frl_apartment001.txt"
traj_est = file_interface.read_tum_trajectory_file(os3path)

In [5]:
tss = traj_est.timestamps/1e9
tss = np.concatenate((np.zeros((1,)), tss),axis=0)
tdiff = tss[1:] - tss[:-1]

for ind,diff in enumerate(tdiff):
    if diff > 10:
        print(ind, diff)

0 1607517702.49489
1038 2237.2170929908752
1894 15.015568256378174


In [6]:
traj_est.timestamps

array([1.60751770e+18, 1.60751770e+18, 1.60751770e+18, ...,
       1.60752025e+18, 1.60752025e+18, 1.60752025e+18])

In [16]:
os3path = "/media/prakhar/BIG_BAG/Capstone/habitat/anand_dec9/f_frl_apartment001.txt"

traj_est1 = file_interface.read_tum_trajectory_file_part(os3path, 0, 1038)
traj_est1.timestamps /= 1e9
traj_est2 = file_interface.read_tum_trajectory_file_part(os3path, 1038, 1894)
traj_est2.timestamps /= 1e9
traj_est3 = file_interface.read_tum_trajectory_file_part(os3path, 1894, len(tss))
traj_est3.timestamps /= 1e9

ls_traj_est = [traj_est1, traj_est2, traj_est3]
ls_traj_ref = [traj_ref1, traj_ref2, traj_ref3]

R = None
t = None
thresh_angle = 20.0
thresh_dist = 0.1
time_temp = 10

for traj_est, traj_ref, gts in zip(ls_traj_est, ls_traj_ref, gt_start):

    offs = -traj_est.timestamps[0] + traj_ref.timestamps[0]
#     traj_ref.transform(lie.se3(M, np.zeros(3)))
    traj_ref, traj_est = sync.associate_trajectories(traj_ref, traj_est, offset_2=offs)
    if R is None and t is None:
        traj_est, R, t, s = trajectory.align_trajectory(traj_est, traj_ref, correct_scale=False, correct_only_scale=False, return_parameters=True)
        print("Aligning the first trajectory, and then will just use that alignment....")
    else:
        traj_est.transform(lie.se3(R, t))
    print(traj_ref)
    print(traj_est)
    print(R)
    print(t)
    print(s)
    diffs = (traj_est.positions_xyz - traj_ref.positions_xyz)
    dists = np.linalg.norm(diffs, axis=1)
    
    print(np.mean(dists), np.std(dists))
    
    ##### Find relocalization time....#####
    tstamps = traj_ref.timestamps
    angle_diffs = np.sum(traj_est.orientations_quat_wxyz * traj_ref.orientations_quat_wxyz, axis=1)
    angle_diffs = 180/np.pi*np.abs(np.pi - np.arccos(2*angle_diffs**2 - 1))
    relocalized = False
    for dist, angle_diff, ts in zip(dists, angle_diffs, traj_est.timestamps):
        if (dist <= thresh_dist) and (angle_diff <= thresh_angle):
            delta = ts - gts
            print("Relocalized in {}s, score={}".format(delta, np.exp(-delta/time_temp)))    
            relocalized = True
            break
        else:
            print("Dists", dist, angle_diff)
    if not relocalized:
        print("Couldn't relocalize!!!! Score=0.0")
#     break


Aligning the first trajectory, and then will just use that alignment....
1038 poses, 24.347m path length, 103.700s duration
1038 poses, 28.189m path length, 103.700s duration
[[ 0.59029847  0.00127398 -0.80718405]
 [-0.00210053 -0.99999294 -0.00311442]
 [-0.80718232  0.00353396 -0.59029163]]
[1.44994303 0.06108897 4.89637818]
1.0
0.04700718163086214 0.013822287764261922
Relocalized in 0.0s, score=1.0
856 poses, 19.277m path length, 85.500s duration
856 poses, 23.285m path length, 85.500s duration
[[ 0.59029847  0.00127398 -0.80718405]
 [-0.00210053 -0.99999294 -0.00311442]
 [-0.80718232  0.00353396 -0.59029163]]
[1.44994303 0.06108897 4.89637818]
1.0
0.06342430827110557 0.03323643739833195
Relocalized in 0.20000004768371582s, score=0.9801986686328038
1033 poses, 19.188m path length, 103.200s duration
1033 poses, 24.825m path length, 103.200s duration
[[ 0.59029847  0.00127398 -0.80718405]
 [-0.00210053 -0.99999294 -0.00311442]
 [-0.80718232  0.00353396 -0.59029163]]
[1.44994303 0.06108

In [ ]:
# traj_est.orientations_quat_wxyz

traj_est.transform(lie.se3(R, t))
traj_est.orientations_quat_wxyz
# [ 0.00510335, -0.89893001,  0.00177525,  0.43805895],
# [ 0.00545427, -0.89899566,  0.0019189 ,  0.43791936],
# [ 0.00553568, -0.89148172,  0.00187331,  0.45301898],

In [ ]:

#####################################STAAAAHPPP##########################
traj_est.timestamps[2], traj_ref.timestamps[2]

In [ ]:
from evo.tools import log
log.configure_logging()
# interactive widgets configuration
import ipywidgets


from evo.tools import plot
from evo.tools.plot import PlotMode
from evo.core.metrics import PoseRelation, Unit
from evo.tools.settings import SETTINGS

# temporarily override some package settings
SETTINGS.plot_figsize = [6, 6]
SETTINGS.plot_split = True
SETTINGS.plot_usetex = False

# magic plot configuration
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib notebook

import evo.main_ape as main_ape
import evo.common_ape_rpe as common

# interactive widgets configuration
import ipywidgets

check_opts_ape = {"align": False, "correct_scale": False, "show_plot": True}
check_boxes_ape=[ipywidgets.Checkbox(description=desc, value=val) for desc, val in check_opts_ape.items()]
check_opts_rpe = {"align": False, "correct_scale": False, "all_pairs": False, "show_plot": True}
check_boxes_rpe=[ipywidgets.Checkbox(description=desc, value=val) for desc, val in check_opts_rpe.items()]
delta_input = ipywidgets.FloatText(value=1.0, description='delta', disabled=False, color='black')
delta_unit_selector=ipywidgets.Dropdown(
    options={u.value: u for u in Unit if u is not Unit.seconds},
    value=Unit.frames, description='delta_unit'
)
plotmode_selector=ipywidgets.Dropdown(
    options={p.value: p for p in PlotMode},
    value=PlotMode.xy, description='plot_mode'
)
pose_relation_selector=ipywidgets.Dropdown(
    options={p.value: p for p in PoseRelation},
    value=PoseRelation.translation_part, description='pose_relation'
)

In [ ]:

count = 0
results = []

def callback_ape(pose_relation, align, correct_scale, plot_mode, show_plot):
    global results, count
    est_name="APE Test #{}".format(count)
    
    result = main_ape.ape(traj_ref, traj_est, est_name=est_name,
                          pose_relation=pose_relation, align=align, correct_scale=correct_scale)
    count += 1
    results.append(result)
    
    if show_plot:
        fig = plt.figure()
        ax = plot.prepare_axis(fig, plot_mode)
        plot.traj(ax, plot_mode, traj_ref, style="--", alpha=0.5)
        plot.traj_colormap(
            ax, result.trajectories[est_name], result.np_arrays["error_array"], plot_mode,
            min_map=result.stats["min"], max_map=result.stats["max"])
    
_ = ipywidgets.interact_manual(callback_ape, pose_relation=pose_relation_selector, plot_mode=plotmode_selector,
                               **{c.description: c.value for c in check_boxes_ape})

In [ ]:
len(traj_ref1.distances)